# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("output_data/cities.csv")
cities_df.head()


,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Port Elizabeth,-33.9180,25.5701,66.20,88,20,13.80,ZA,2021-03-12 03:03:14
1,Sitka,57.0531,-135.3300,35.60,64,90,6.69,US,2021-03-12 03:03:16
2,Hilo,19.7297,-155.0900,75.20,83,90,9.22,US,2021-03-12 03:03:17
3,Celestún,20.8667,-90.4000,76.69,73,3,17.60,MX,2021-03-12 03:03:18
4,Butaritari,3.0707,172.7902,81.91,74,0,13.24,KI,2021-03-12 03:03:19


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [8]:
locations = cities_df[["Latitude", "Longitude"]]

humidity_as_weight = cities_df["Humidity"]

fig = gmaps.figure(zoom_level=1, center=(0, 0))

# Adding Heatmap layer to map
heat_layer = gmaps.heatmap_layer(locations, weights=humidity_as_weight, 
                                 dissipating=False, max_intensity=1000,
                                 point_radius = 5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
